In [11]:
    """
        BERT-pytorch의 corpus 제작
    """
    import json
    import os
    from pre_train import train_tokenizer
    import utils

    from transformers import BertTokenizer
    from plm_dataset import PLMDataset


    config_path = "/root/data/ojt/config/bert_debug.json"
    output_dir = "/root/data/ojt/reference/BERT-pytorch/dataset"
    train_path = "/root/data/ojt/datasets/book_corpus_debug.txt"

    # config_path = "/root/data/ojt/config/bert_debug.json"
    # output_dir = "/root/data/ojt/reference/BERT-pytorch/dataset/books_large_p1_25"
    # train_path = "/root/data/ojt/datasets/books_large_p1_25.txt"
    

    output_path=os.path.join(output_dir,os.path.basename(train_path))
    vocab_file_path = os.path.join(output_dir,utils.VOCAB_TXT_FILE_NAME)

    with open(config_path,"r") as cfg_json:
        config = json.load(cfg_json)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    train_tokenizer(train_path,output_dir,vocab_size=1000)

    
    tokenizer=BertTokenizer(vocab_file=vocab_file_path,do_lower_case=True)
    id_to_vocab={v:k for k,v in tokenizer.get_vocab().items()}
    # print(id_to_vocab)

    plm_dataset=PLMDataset(train_path,tokenizer,config["max_seq_len"],config["max_mask_tokens"],cached_dir=output_dir,use_cache=False)

    with open(output_path,"w") as f:
        first_data=True
        
        for data in plm_dataset:
            # print(data)
            if first_data:
                first_data=False
            else:
                f.write("\n")
            first_input=True
            mask_index=0
            mask_label_list = data["mlm_labels"].tolist()
            mlm_positions = data["mlm_positions"].tolist()
            print(mlm_positions)
            for index, input_id in enumerate(data["input_ids"].tolist()):
                print("input=",id_to_vocab[input_id])
                if not first_input:
                    f.write(" ")
                else:
                    first_input=False

                if index in mlm_positions and index!=0:
                    print(index)
                    print(id_to_vocab[mask_label_list[mask_index]])
                    mask_index+=1
                vocab=id_to_vocab[input_id]
                if vocab=="[SEP]":
                    f.write("\t")
                elif vocab=="[PAD]":
                    break
                elif vocab=="[MASK]":

                    # print(mask_label_list[mask_index])?

                    f.write(id_to_vocab[mask_label_list[mask_index]])

                else:
                    f.write(vocab)
            


    print("Written to %s"%output_path)




Start Loading training data
loading 0/31
Training data successfully loaded
Writing pkl file = /root/data/ojt/reference/BERT-pytorch/dataset/book_corpus_debug.pkl
Successfully written
doc num = 1, sentence num = 31
[2, 5, 10, 21, 27, 32, 36, 42, 61, 63, 64, 68, 75, 85, 105, 106, 107, 123, 0, 0]
input= [CLS]
input= i
input= wi
2
wi
input= ##s
input= ##h
input= [MASK]
5
i
input= ha
input= ##d
input= a
input= be
input= [MASK]
10
##t
input= ##t
input= ##er
input= an
input= ##s
input= ##w
input= ##er
input= to
input= that
input= q
input= ##u
input= [MASK]
21
##es
input= ##t
input= ##ion
input= .
input= star
input= ##l
input= [MASK]
27
##ings
input= ,
input= ne
input= ##w
input= y
input= [MASK]
32
##or
input= ##k
input= is
input= no
input= [MASK]
36
##t
input= the
input= place
input= you
input= ##d
input= e
input= [MASK]
42
##x
input= ##p
input= ##e
input= ##c
input= ##t
input= m
input= ##u
input= ##ch
input= to
input= ha
input= ##pp
input= ##en
input= .
input= [SEP]
input= its
input= a
in